In [ ]:
# Setup: Ensure the animal_shelter module is in the Python path
import sys
from pathlib import Path

candidate = Path.cwd()
project_root = None
# Search up for the project root to ensure module can be imported
for _ in range(6):
    if (candidate / "animal_shelter").is_dir():
        project_root = candidate
        break
    candidate = candidate.parent

if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [ ]:
# Import necessary libraries
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd

# Import the CRUD module
from animal_shelter.animal_shelter import AnimalShelter

# --- App Styling ---
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(__name__, external_stylesheets=external_stylesheets)

# --- App Layout ---
app.layout = html.Div([
    html.Div(className='container',
        children=[
            html.H1("Dave Mobley's CS 340 Dashboard", style={'textAlign': 'center'}),
            html.P("Enter credentials to connect to the AAC database and query for 'Lucy' the 'Dog'.", style={'textAlign': 'center'}),
            
            # Input and Button Div
            html.Div([
                dcc.Input(
                    id="input_user", type="text", placeholder="Username",
                    style={'marginRight': '10px', 'padding': '10px', 'fontSize': '16px'}
                ),
                dcc.Input(
                    id="input_passwd", type="password", placeholder="Password",
                    style={'marginRight': '10px', 'padding': '10px', 'fontSize': '16px'}
                ),
                html.Button('Login and Query', id='submit-val', n_clicks=0, style={'padding': '10px', 'fontSize': '16px'})
            ], style={'display': 'flex', 'justifyContent': 'center', 'padding': '20px'}),
            
            html.Hr(),
            
            # Output Area for Query Results
            html.Div(id="query-out")
        ]
    )
]) # End of main Div

# --- Callback to Update Data Table ---
@app.callback(
    Output('query-out', 'children'),
    Input('submit-val', 'n_clicks'),
    [Input('input_user', 'value'),
     Input('input_passwd', 'value')]
)
def update_output(n_clicks, username, password):
    if n_clicks == 0:
        return html.Div()

    if not username or not password:
        return html.P("Error: Please enter both a username and password.", style={'color': 'red', 'textAlign': 'center'})

    try:
        # Instantiate CRUD object with user authentication
        shelter = AnimalShelter(user=username, password=password)
        
        # Define the query
        query = {"animal_type": "Dog", "name": "Lucy"}
        
        # Execute read operation
        results = shelter.read(query)
        
        if not results:
            return html.P(f"Query returned no documents for: {query}", style={'textAlign': 'center'})
        
        # Convert results to a pandas DataFrame
        df = pd.DataFrame(results)
        # Remove the internal MongoDB ID for a cleaner display
        if '_id' in df.columns:
            df = df.drop(columns=['_id'])
        
        # Generate a styled Plotly table
        table = go.Figure(data=[go.Table(
            header=dict(values=list(df.columns), fill_color='royalblue', font=dict(color='white', size=14), align='left'),
            cells=dict(values=[df[col] for col in df.columns], fill_color='lavender', align='left', font_size=12)
        )])
        table.update_layout(title='Query Results')
        return dcc.Graph(figure=table)

    except Exception as e:
        return html.P(f"Error: Could not connect or query database. Please check credentials. Details: {e}", style={'color': 'red', 'textAlign': 'center'})

# --- Run the App ---
# Use jupyter_mode='external' to get a URL that you can open in a new tab.
app.run(jupyter_mode='external')